<div style="background-color: #87CEEB; border-radius: 12px; padding: 20px; border: 3px solid #87CEEB;">
    <strong style="color: #000000; font-size: 1.5em;">MODELOS DE PREDICCIÓN:</strong>
</div>

In [1]:
# Importaciones necesarias para el correcto funcionamiento de todos lo modelos y demás operaciones con los datos
import importlib
import joblib
import subprocess
import warnings
import numpy as np
import pandas as pd
import sklearn
import lightgbm as lgb


!pip install session_info

import session_info

#Froms
from keras.layers import Dense, LSTM
from keras.models import Sequential
from packaging import version
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestRegressor, VotingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import make_scorer, r2_score
from sklearn.model_selection import GridSearchCV, KFold, cross_val_score, train_test_split
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from xgboost import XGBRegressor

C:\Users\34634\anaconda3\lib\site-packages\pandas\core\computation\expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
C:\Users\34634\anaconda3\lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
result_df = pd.read_csv("../Fitabase Data 4.12.16-5.12.16/test_train_data.csv")
result_df

,Id,Time,HeartRate,Intensity,Calories
0,2022484408,2016-04-12 07:21:00,101.600000,1,3.32064
1,2022484408,2016-04-12 07:22:00,87.888889,1,3.94326
2,2022484408,2016-04-12 07:23:00,58.000000,0,1.34901
3,2022484408,2016-04-12 07:24:00,58.000000,0,1.03770
4,2022484408,2016-04-12 07:25:00,56.777778,0,1.03770
...,...,...,...,...,...
333141,8877689391,2016-05-12 13:55:00,60.666667,0,1.33353
333142,8877689391,2016-05-12 13:56:00,61.875000,0,1.33353
333143,8877689391,2016-05-12 13:57:00,58.142857,0,1.33353
333144,8877689391,2016-05-12 13:58:00,61.200000,0,1.33353



<div style="background-color: #87CEEB; border-radius: 12px; padding: 20px; border: 3px solid #87CEEB;">
    <strong style="color: #000000; font-size: 1.5em;"> Instalación/Actualización de librerías necesarias:📦</strong>
</div>


In [3]:
try:
    importlib.import_module('xgboost')
    print("XGBoost ya está instalado en el sistema.")
except ImportError:
    subprocess.run(['pip', 'install', 'xgboost'])

XGBoost ya está instalado en el sistema.


In [4]:
try:
    importlib.import_module('lightgbm')
    print("LightGBM ya está instalado en el sistema.")
except ImportError:
    subprocess.run(['pip', 'install', 'lightgbm'])

LightGBM ya está instalado en el sistema.


In [5]:
try:
    importlib.import_module('tensorflow')
    print("TensorFlow ya está instalado en el sistema.")
except ImportError:
    subprocess.run(['pip', 'install', 'tensorflow', 'scikit-learn'])

TensorFlow ya está instalado en el sistema.


In [6]:
subprocess.run(['pip', 'install', '--upgrade', 'lightgbm', 'pandas', 'dask'])

CompletedProcess(args=['pip', 'install', '--upgrade', 'lightgbm', 'pandas', 'dask'], returncode=0)

In [7]:
required_version = '14.0.1'

try:
    pyarrow_version = importlib.import_module('pyarrow').__version__
    if version.parse(pyarrow_version) < version.parse(required_version):
        print(f"PyArrow versión {pyarrow_version} encontrada. Actualizando a la versión {required_version}.")
        subprocess.run(['pip', 'install', f'pyarrow>={required_version}'])
    else:
        print(f"PyArrow versión {pyarrow_version} ya está instalada.")
except ImportError:
    subprocess.run(['pip', 'install', f'pyarrow>={required_version}'])


PyArrow versión 15.0.0 ya está instalada.


<div style="background-color: #8990EB; border-radius: 12px; padding: 20px; border: 3px solid #8990EB;">
    <strong style="color: #000000; font-size: 1.5em;">⚠ **User Alert:**</strong> Don't forget to RESTART the kernel  <strong style="color: #000000; font-size: 1em;">(if libraries were not installed in current PC)</strong> for the magic to happen after ADOPTING the new libraries.
</div>

<div style="background-color: #87CEEB; border-radius: 12px; padding: 20px; border: 3px solid #87CEEB;">
    <strong style="color: #000000; font-size: 1.5em;">Información de sesión:</strong>
</div>

In [8]:
session_info.show(html=False)

-----
joblib              1.3.2
keras               2.15.0
lightgbm            4.4.0
numpy               1.24.4
packaging           21.3
pandas              2.2.2
session_info        1.0.0
sklearn             1.0.2
xgboost             2.0.3
-----
IPython             8.18.1
jupyter_client      7.3.4
jupyter_core        4.11.1
jupyterlab          3.4.4
notebook            6.4.12
-----
Python 3.9.13 (main, Aug 25 2022, 23:51:50) [MSC v.1916 64 bit (AMD64)]
Windows-10-10.0.22000-SP0
-----
Session information updated at 2024-07-18 09:23


<div style="background-color: #90EE90; border-radius: 12px; padding: 20px; border: 3px solid #90EE90;">
    <strong style="color: #000000; font-size: 1.5em;">Preprocesamiento de los datos:🔧</strong>
</div>

In [9]:
result_df['Time'] = pd.to_datetime(result_df['Time'])

result_df['Hour'] = result_df['Time'].dt.hour
result_df['Minutes'] = result_df['Time'].dt.minute
result_df['Weekday'] = result_df['Time'].dt.weekday

features = ['Id', 'Hour', 'Minutes', 'Intensity', 'Calories']
target = 'HeartRate'

X = result_df[features]
y = result_df[target]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.17, random_state=777)

categorical_features = ['Id']
numeric_features = ['Hour', 'Minutes', 'Intensity', 'Calories']

preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_features),
        ('cat', OneHotEncoder(), categorical_features)
    ], remainder='passthrough')

<div style="background-color: #D8BFD9; border-radius: 12px; padding: 20px; border: 3px solid #D8BFD8;">
    <strong style="color: #000000; font-size: 1.5em;">Linear Regression</strong>
</div>

In [10]:
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', LinearRegression())
])

pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)
r2 = r2_score(y_test, y_pred)
print(f"\033[94;1mR2 score: {r2}\033[0m")

R2 score: 0.7415176189275107


<div style="background-color: #D8BFD9; border-radius: 12px; padding: 20px; border: 3px solid #D8BFD8;">
    <strong style="color: #000000; font-size: 1.5em;">Gradient Boosted Trees : XGBoost🌳</strong>
</div>

In [18]:
num_splits = 5 
xgb_model = XGBRegressor(objective='reg:squarederror', random_state=42)
xgb_scores = cross_val_score(xgb_model, X, y, scoring='r2', cv=KFold(n_splits=num_splits, shuffle=True, random_state=42))

print(f"\033[94;1mPromedio R2 Score (XGBoost): {xgb_scores.mean()}\033[0m")

Promedio R2 Score (XGBoost): 0.858344518126301


In [11]:
best_params = {'colsample_bytree': 1.0, 'learning_rate': 0.2, 'max_depth': 7, 'n_estimators': 200, 'subsample': 1.0}

# Crear el modelo XGBoost con los mejores hiperparámetros
optimal_model = XGBRegressor(
    objective='reg:squarederror',
    random_state=42,
    n_estimators=best_params['n_estimators'],
    learning_rate=best_params['learning_rate'],
    max_depth=best_params['max_depth'],
    subsample=best_params['subsample'],
    colsample_bytree=best_params['colsample_bytree']
)

num_splits = 5
xgb_scores = cross_val_score(optimal_model, X, y, scoring='r2', cv=KFold(n_splits=num_splits, shuffle=True, random_state=42))
print(f"\033[94;1mPromedio R2 Score (XGBoost) con mejores hiperparámetros: {xgb_scores.mean()}\033[0m")

Promedio R2 Score (XGBoost) con mejores hiperparámetros: 0.8604436463886229


<div style="background-color: #D8BFD8; border-radius: 12px; padding: 20px; border: 3px solid #D8BFD8;">
    <strong style="color: #000000; font-size: 1.5em;">LightGBM:🌳</strong>
</div>

In [32]:
best_params = {'learning_rate': 0.2, 'n_estimators': 200, 'num_leaves': 40}
best_model = lgb.LGBMRegressor(objective='regression', random_state=42, force_row_wise=True, **best_params)

num_splits = 5 
lgb_scores = cross_val_score(best_model, X, y, scoring='r2', cv=KFold(n_splits=num_splits, shuffle=True, random_state=42))

print(f"\033[94;1mPromedio R2 Score (LightGBM): {lgb_scores.mean()}\033[0m")

C:\Users\34634\anaconda3\lib\site-packages\joblib\externals\loky\backend\context.py:150: UserWarning: Could not find the number of physical cores for the following reason:
[WinError 2] El sistema no puede encontrar el archivo especificado
Returning the number of logical cores instead. You can silence this warning by setting LOKY_MAX_CPU_COUNT to the number of cores you want to use.
  warnings.warn(
  File "C:\Users\34634\anaconda3\lib\site-packages\joblib\externals\loky\backend\context.py", line 227, in _count_physical_cores
    cpu_info = subprocess.run(
  File "C:\Users\34634\anaconda3\lib\subprocess.py", line 505, in run
    with Popen(*popenargs, **kwargs) as process:
  File "C:\Users\34634\anaconda3\lib\subprocess.py", line 951, in __init__
    self._execute_child(args, executable, preexec_fn, close_fds,
  File "C:\Users\34634\anaconda3\lib\subprocess.py", line 1420, in _execute_child
    hp, ht, pid, tid = _winapi.CreateProcess(executable, args,


[LightGBM] [Info] Total Bins 358
[LightGBM] [Info] Number of data points in the train set: 266516, number of used features: 5
[LightGBM] [Info] Start training from score 73.702794
[LightGBM] [Info] Total Bins 358
[LightGBM] [Info] Number of data points in the train set: 266517, number of used features: 5
[LightGBM] [Info] Start training from score 73.701315
[LightGBM] [Info] Total Bins 358
[LightGBM] [Info] Number of data points in the train set: 266517, number of used features: 5
[LightGBM] [Info] Start training from score 73.703051
[LightGBM] [Info] Total Bins 358
[LightGBM] [Info] Number of data points in the train set: 266517, number of used features: 5
[LightGBM] [Info] Start training from score 73.734438
[LightGBM] [Info] Total Bins 358
[LightGBM] [Info] Number of data points in the train set: 266517, number of used features: 5
[LightGBM] [Info] Start training from score 73.707150
Promedio R2 Score (LightGBM): 0.8608125059372875


In [33]:
joblib.dump(best_model, '../ML models/LightGBM_ML.pkl')

['../ML models/LightGBM_ML.pkl']

In [13]:
# Mejores hiperparámetros encontrados previamente con GridSearch
best_params = {'learning_rate': 0.2, 'n_estimators': 200, 'num_leaves': 40}

best_model = lgb.LGBMRegressor(objective='regression', random_state=42, **best_params)

num_splits = 5 
lgb_params = best_params.copy()
lgb_params['force_row_wise'] = True

best_model_force_row_wise = lgb.LGBMRegressor(objective='regression', random_state=42, **lgb_params)
pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                             ('regressor', best_model_force_row_wise)])
lgb_scores = cross_val_score(pipeline, X, y, scoring='r2', cv=KFold(n_splits=num_splits, shuffle=True, random_state=42))

print(f"\033[94;1mPromedio R2 Score (LightGBM): {lgb_scores.mean()}\033[0m")

[LightGBM] [Info] Total Bins 374
[LightGBM] [Info] Number of data points in the train set: 266516, number of used features: 18
[LightGBM] [Info] Start training from score 73.702794
[LightGBM] [Info] Total Bins 374
[LightGBM] [Info] Number of data points in the train set: 266517, number of used features: 18
[LightGBM] [Info] Start training from score 73.701315
[LightGBM] [Info] Total Bins 374
[LightGBM] [Info] Number of data points in the train set: 266517, number of used features: 18
[LightGBM] [Info] Start training from score 73.703051
[LightGBM] [Info] Total Bins 374
[LightGBM] [Info] Number of data points in the train set: 266517, number of used features: 18
[LightGBM] [Info] Start training from score 73.734438
[LightGBM] [Info] Total Bins 374
[LightGBM] [Info] Number of data points in the train set: 266517, number of used features: 18
[LightGBM] [Info] Start training from score 73.707150
Promedio R2 Score (LightGBM): 0.8555762087754732


<div style="background-color: #D8BFD8; border-radius: 12px; padding: 20px; border: 3px solid #D8BFD8;">
    <strong style="color: #000000; font-size: 1.5em;">LSTM: Long Short Term Memory</strong>
</div>

In [14]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train[numeric_features])
X_test_scaled = scaler.transform(X_test[numeric_features])
X_train_reshaped = X_train_scaled.reshape((X_train_scaled.shape[0], 1, X_train_scaled.shape[1]))
X_test_reshaped = X_test_scaled.reshape((X_test_scaled.shape[0], 1, X_test_scaled.shape[1]))


model = Sequential()
model.add(LSTM(120, activation='relu', input_shape=(1, len(numeric_features))))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')


model.fit(X_train_reshaped, y_train, epochs=15, batch_size=32, verbose=1)

y_pred = model.predict(X_test_reshaped)


r2 = r2_score(y_test, y_pred)
print(f"\033[94;1mR2 Score: {r2:.6f}\033[0m")




Epoch 1/15

8641/8641 [==============================] - 25s 3ms/step - loss: 389.5086
Epoch 2/15
8641/8641 [==============================] - 24s 3ms/step - loss: 119.9024
Epoch 3/15
8641/8641 [==============================] - 25s 3ms/step - loss: 118.6448
Epoch 4/15
8641/8641 [==============================] - 26s 3ms/step - loss: 117.6529
Epoch 5/15
8641/8641 [==============================] - 25s 3ms/step - loss: 116.6633
Epoch 6/15
8641/8641 [==============================] - 26s 3ms/step - loss: 115.6028
Epoch 7/15
8641/8641 [==============================] - 28s 3ms/step - loss: 114.7307
Epoch 8/15
8641/8641 [==============================] - 29s 3ms/step - loss: 114.0143
Epoch 9/15
8641/8641 [==============================] - 26s 3ms/step - loss: 113.6048
Epoch 10/15
8641/8641 [==============================] - 26s 3ms/step - loss: 113.2977
Epoch 11/15
8641/8641 [==============================] - 25s 3ms/step - loss: 113.1068
Epoch 12/15
8641/8641 [==========================

<div style="background-color: #D8BFD8; border-radius: 12px; padding: 20px; border: 3px solid #D8BFD8;">
    <strong style="color: #000000; font-size: 1.5em;">MLPRegressor: Multi-Layer Perceptron Regressor</strong>
</div>

In [10]:
pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                             ('regressor', MLPRegressor(random_state=42, max_iter=1300))])
num_splits = 5  

# Validación cruzada 
mlp_scores = cross_val_score(pipeline, X, y, scoring='r2', cv=KFold(n_splits=num_splits, shuffle=True, random_state=42))

print(f"R2 Scores (MLPRegressor) en {num_splits}-fold cross-validation: {mlp_scores}")
print(f"\033[94;1mPromedio R2 Score (MLPRegressor): {mlp_scores.mean()}\033[0m")

R2 Scores (MLPRegressor) en 5-fold cross-validation: [0.85287847 0.84818361 0.84978637 0.84772851 0.85042114]
Promedio R2 Score (MLPRegressor): 0.8497996193926494


<div style="background-color: #D8BFD8; border-radius: 12px; padding: 20px; border: 3px solid #D8BFD8;">
    <strong style="color: #000000; font-size: 1.5em;">KNN Regressor:📍</strong>
</div>

In [29]:
numeric_features = ['Hour', 'Minutes', 'Intensity', 'Calories']
categorical_features = ['Id']


model = KNeighborsRegressor(n_neighbors=5, algorithm='brute')

pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('model', model)
])

num_splits = 5
knn_scores = cross_val_score(pipeline, X, y, scoring='r2', cv=KFold(n_splits=num_splits, shuffle=True, random_state=42))

print(f"\033[94;1mPPromedio R2 Score (KNeighborsRegressor): {knn_scores.mean()}\033[0m")

PPromedio R2 Score (KNeighborsRegressor): 0.8261730442623068


<div style="background-color: #D8BFD8; border-radius: 12px; padding: 20px; border: 3px solid #D8BFD8;">
    <strong style="color: #000000; font-size: 1.5em;">Decision Tree Regressor:🌳</strong>
</div>

In [20]:
dt_regressor = DecisionTreeRegressor(random_state=11101)


pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', dt_regressor)
])

num_splits = 5
scores = cross_val_score(pipeline, X_train, y_train, cv=num_splits, scoring='neg_mean_squared_error')
mse_scores = -scores 

pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)


mse_test = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error on Test Set: {mse_test}')


print(f'Mean MSE across {num_splits}-fold Cross-Validation: {mse_scores.mean()}')
print(f'\033[94;1mMSE Scores for each fold: {mse_scores}\033[0m')

Mean Squared Error on Test Set: 60.792109505792425
Mean MSE across 5-fold Cross-Validation: 62.69339791385077
MSE Scores for each fold: [62.22259701 62.84575426 63.5455043  62.57267111 62.28046288]


<div style="background-color: #D8BFD8; border-radius: 12px; padding: 20px; border: 3px solid #D8BFD8;">
    <strong style="color: #000000; font-size: 1.5em;">SVM: Support Vector Machine Regressor:🤖</strong>
</div>

In [ ]:
model = SVR(kernel='rbf', gamma='auto', epsilon=0.1)

model.fit(X_train, y_train)

y_pred = model.predict(X_test)

score = r2_score(y_test, y_pred)
print("R2 score on the test set:", score)

# Cross-validation
num_splits = 5
svm_scores = cross_val_score(model, X, y, scoring='r2', cv=KFold(n_splits=num_splits, shuffle=True, random_state=11101))

print(f"R2 Scores (SVR) in {num_splits}-fold cross-validation: {svm_scores}")
print(f"\033[94;1mAverage R2 Score (SVR) across {num_splits}-fold cross-validation: {svm_scores.mean()}\033[0m")

<div style="background-color: #D8BFD8; border-radius: 12px; padding: 20px; border: 3px solid #D8BFD8;">
    <strong style="color: #000000; font-size: 1.5em;">Random Forest Regressor:🌳</strong>
</div>

In [ ]:
model = RandomForestRegressor(n_estimators=100, random_state=11101)


pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('model', model)
])

pipeline.fit(X_train, y_train)
score = pipeline.score(X_test, y_test)
print(f'R2 Score on Test Set: {score}')

num_splits = 5
rf_scores = cross_val_score(pipeline, X, y, scoring='r2', cv=KFold(n_splits=num_splits, shuffle=True, random_state=11101))

print(f"R2 Scores (Random Forest) in {num_splits}-fold cross-validation: {rf_scores}")
print(f"\033[94;1mAverage R2 Score (Random Forest) across {num_splits}-fold cross-validation: {rf_scores.mean()}\033[0m")

<div style="background-color: #D8BFD8; border-radius: 12px; padding: 20px; border: 3px solid #D8BFD8;">
    <strong style="color: #000000; font-size: 1.5em;"> Modelo Combinado: LightGBM, RandomForest, XGBoost</strong>
</div>

In [11]:
best_params = {'learning_rate': 0.2, 'n_estimators': 200, 'num_leaves': 31, 'max_depth': 6, 'subsample': 0.8, 'colsample_bytree': 0.8}


xgb_model = XGBRegressor(
    objective='reg:squarederror',
    random_state=42,
    n_estimators=best_params['n_estimators'],
    learning_rate=best_params['learning_rate'],
    max_depth=best_params['max_depth'],
    subsample=best_params['subsample'],
    colsample_bytree=best_params['colsample_bytree']
)

lgb_model = lgb.LGBMRegressor(
    objective='regression',
    random_state=42,
    learning_rate=best_params['learning_rate'],
    n_estimators=best_params['n_estimators'],
    num_leaves=best_params['num_leaves']
)

rf_model = RandomForestRegressor(random_state=42)

voting_regressor = VotingRegressor(
    estimators=[('xgb', xgb_model), ('lgb', lgb_model), ('rf', rf_model)]
)
voting_regressor.fit(X_train, y_train)

score = voting_regressor.score(X_test, y_test)
print(f"\033[94;1mR2 score del modelo combinado en el conjunto de prueba: {score}\033[0m")

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.003243 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 358
[LightGBM] [Info] Number of data points in the train set: 276511, number of used features: 5
[LightGBM] [Info] Start training from score 73.705764
R2 score del modelo combinado en el conjunto de prueba: 0.8641493240621558


In [12]:
joblib.dump(voting_regressor, '../ML models/L_XG_RF_ML.pkl')

['ML models/L_XG_RF_ML.pkl']